In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import celltypist
import matplotlib.pyplot as plt
from pyscenic.export import add_scenic_metadata
from pyscenic.cli.utils import load_signatures
import seaborn as sns
from matplotlib import cm, colors
from matplotlib.collections import LineCollection

Prepare data for running SCENIC.

In [ ]:
##in vitro data from organoids
adata_vitro = sc.read('../../../scvi_remove_lowUMI/HVG_1000_latent_10_res.h5ad')
adata_vitro = adata_vitro.raw.to_adata()
ind = ~adata_vitro.obs['celltype'].str.startswith('TOM_').values
adata_vitro = adata_vitro[ind, :]

adata2 = sc.read('../../../ogn_raw_counts_all_cells.h5ad')
adata2 = adata2[adata_vitro.obs_names, :]
adata2.obs = adata_vitro.obs.copy()

##subsampling 500 cells in each cell type
sdata = celltypist.samples.downsample_adata(adata2, mode='each', by='celltype', n_cells=500, return_index=False)
sc.pp.filter_genes(sdata, min_cells=20)

adata2[:, sdata.var_names].to_df().to_csv('organoid_EVTM_all_cells_exp.csv')
sdata.to_df().to_csv('organoid_EVTM_subsample_500_exp.csv')

##in vivo data from Arutyunyan et al., Nature 2023
adata_vivo = sc.read('../../../adata_P13_trophoblast_raw_counts_in_raw_normlog_counts_in_X_for_download.h5ad')
adata_vivo = adata_vivo.raw.to_adata()
adata_vivo.to_df().to_csv('invivo_all_cells_exp.csv')

Run SCENIC using the CLI version.

In [ ]:
!singularity run aertslab-pyscenic-0.12.1.sif pyscenic grn -o pyscenic_output/adjacencies.tsv --seed 998 --num_workers 20 organoid_EVTM_subsample_500_exp.csv pyscenic_data/allTFs_hg38.txt
_hg38.txt

!singularity run aertslab-pyscenic-0.12.1.sif pyscenic ctx -o pyscenic_output/regulons.csv --annotations_fname pyscenic_data/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl --num_workers 20 --min_genes 10 --expression_mtx_fname organoid_EVTM_subsample_500_exp.csv --mask_dropouts pyscenic_output/adjacencies.tsv pyscenic_data/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather pyscenic_data/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather

##run aucell for in vitro cells
!singularity run aertslab-pyscenic-0.12.1.sif pyscenic aucell -o pyscenic_output/auc_mtx.csv --num_workers 20 --seed 998 organoid_EVTM_all_cells_exp.csv pyscenic_output/regulons.csv

##run aucell for in vivo cells
!singularity run aertslab-pyscenic-0.12.1.sif pyscenic aucell -o pyscenic_output/auc_mtx_invivo.csv --num_workers 20 --seed 998 invivo_all_cells_exp.csv pyscenic_output/regulons.csv

Check the enrichment of the derived regulons in molecular changes induced by uNK cytokines.

In [ ]:
auc_mtx = pd.read_csv('pyscenic_output/auc_mtx.csv', index_col=0)
sig = load_signatures('pyscenic_output/regulons.csv')
adata_vitro = add_scenic_metadata(adata_vitro, auc_mtx, sig)

ggs = pd.read_csv('../../../EVT_invivo_invitro_DE_gene_overlap_up.csv', header=None)[0].values
n1 = len(ggs)
n2 = pd.read_csv('organoid_EVTM_subsample_500_exp.csv', index_col=0, nrows=2).shape[1] - n1

rgls = adata_vitro.var.columns[3:]
rgls_nm = rgls.str.replace('Regulon(', '').str.replace('(+))', '')
num = []
for rgl in rgls:
    n3 = adata_vitro.var[rgl].sum()
    n4 = len(adata_vitro.var.index[adata_vitro.var[rgl]].intersection(ggs))
    num += [[n4, n1, n2, n3]]
pd.DataFrame(num, index=rgls_nm).to_csv('check_scenic_res_enrichment.csv', header=False)

##The following code was run in R instead of python
dd <- read.csv('check_scenic_res_enrichment.csv', row.names=1, stringsAsFactors=F, header=F)
pval <- apply(dd, 1, function(x) 1 - phyper(x[1]-1, x[2], x[3], x[4]))
pval_adj <- p.adjust(pval, method='BH')
write.csv(data.frame(pval_adj), file='check_scenic_res_enrichment_hyper_res.csv', row.names=T, col.names=F, quote=F)

The final TFs were determined based on the following criteria: 1) its targets were significantly enriched in the genes upregulated by uNK cytokines (BH-corrected p-value < 0.05); 2) it showed a significant increase in expression level after exposure to cytokines in at least one of the VCT and EVT subtypes (Bonferroni-corrected p-value < 0.05; expression percent in cytokine-treated cells > 0.1); 3) the average AUCell score for the regulon across cells from the EVT subtype ‘EVT_late_3’ was > 0.05. The AUCell scores for these regulons were then visualised along the pseudotime estimated for the in vitro and in vivo EVT.

In [ ]:
##add SCENIC result into adata from organoids
adata_vitro = sc.read('../../../scvi_remove_lowUMI/HVG_1000_latent_10_res.h5ad')
adata_vitro = adata_vitro.raw.to_adata()
ind = ~adata_vitro.obs['celltype'].str.startswith('TOM_').values
adata_vitro = adata_vitro[ind, :].copy()
auc_mtx = pd.read_csv('../../../CSC_revision/pyscenic_output/auc_mtx.csv', index_col=0)
sig = load_signatures('../../../CSC_revision/pyscenic_output/regulons.csv')
adata_vitro = add_scenic_metadata(adata_vitro, auc_mtx, sig)
pdata_vitro = sc.read('../../../organoid_EVT_sctour_prediction_res.h5ad')
adata_vitro = adata_vitro[pdata_vitro.obs_names].copy()
adata_vitro.obs['ptime'] = pdata_vitro.obs['ptime'].copy()

##add SCENIC result into adata from primary tissue
adata_vivo = sc.read('../../../adata_P13_trophoblast_raw_counts_in_raw_normlog_counts_in_X_for_download.h5ad')
adata_vivo = adata_vivo.raw.to_adata()
adata_vivo.obs['celltype'] = adata_vivo.obs['final_annot_all_troph_corrected'].copy()
auc_mtx_vivo = pd.read_csv('../../../CSC_revision/pyscenic_output/auc_mtx_invivo.csv', index_col=0)
adata_vivo = add_scenic_metadata(adata_vivo, auc_mtx_vivo, sig)
pdata_vivo = sc.read('../../../Roser_new_data_sctour_res.h5ad')
adata_vivo = adata_vivo[pdata_vivo.obs_names].copy()
adata_vivo.obs['ptime'] = pdata_vivo.obs['ptime'].copy()

##select regulons
ctys = ['VCT', 'EVT_proliferating', 'EVT_early_1', 'EVT_early_2','EVT_early_3',
        'EVT_intermediate_1', 'EVT_intermediate_2',
        'EVT_late_1', 'EVT_late_2', 'EVT_late_3']
eres = pd.read_csv('../../../CSC_revision/check_scenic_res_enrichment_hyper_res.csv', index_col=0)
ggs = eres.index[eres['pval_adj'] < 0.05]
pval = pd.read_csv('../../../cytokine_nocytokine_pval.csv', index_col=0)[ctys]
lgfd = pd.read_csv('../../../cytokine_nocytokine_lgfd.csv', index_col=0)[ctys]
pct = pd.read_csv('../../../cytokine_nocytokine_pct1.csv', index_col=0)[ctys]
ind1 = (pval.loc[ggs] < 0.05)
ind2 = (lgfd.loc[ggs] > 0)
ind3 = (pct.loc[ggs] > 0.1)
ind4 = (adata_vitro.obs.loc[(adata_vitro.obs['celltype']=='EVT_late_3').values, [f'Regulon({gg}(+))' for gg in ggs]].mean() > 0.05)
ind = (((ind1 & ind2 & ind3).sum(1) > 0).values & ind4.values)
ggs = ggs[ind]
print(ggs)

##visualize their AUCell scores along the EVT differentiation pathway in vitro and in vivo
cols = list(map(colors.to_hex, cm.tab20.colors))
cols2 = list(map(colors.to_hex, cm.tab20b.colors))
cols3 = list(map(colors.to_hex, cm.tab20c.colors))

ctys1 = ctys[1:]
ctys2 = ['VCT_CCC', 'EVT_1', 'EVT_2', 'iEVT', 'GC']
ctys = [ctys1, ctys2]
col1 = [cols[18]] + [cols3[11]] + [cols[1]] + [cols[19]] + 
       ['#1d91c0'] + ['#4a6fe3'] +
       [cols2[2]] + ['#023fa5'] + ['#0000ff']
col2 = ['#fb9a99', '#fdbf6f', '#f768a1', '#ff7f00', '#e31a1c']
pcols = [col1, col2]

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(10, 2), gridspec_kw=dict(bottom=0.1, top=0.9))
for i, adata in enumerate([adata_vitro, adata_vivo]):
    adata = adata[np.argsort(adata.obs['ptime'].values), :].copy()
    adata.obs['celltype'] = adata.obs['celltype'].astype('category')
    adata.obs['celltype'] = adata.obs['celltype'].cat.reorder_categories(ctys[i])
    adata.uns['celltype_colors'] = pcols[i]
    
    mat = adata.obs[[f'Regulon({gg}(+))' for gg in ggs]].values
    mm = mat.mean(0)
    std = np.std(mat, axis=0)
    mat = (mat - mm)/std
    mat[mat > 10] = 10
    mat = mat.T
    weights = np.ones(30) / 30
    for m in range(mat.shape[0]):
        mat[m] = np.convolve(mat[m], weights, mode="same")
        
    laxs = axs[i].get_subplotspec().subgridspec(nrows=3, ncols=2, height_ratios=[0.08, 0.08, 1], width_ratios=[1, 0.04], hspace=0.02)
    axs[i].remove()
    ax1 = fig.add_subplot(laxs[2, 0])
    ax2 = fig.add_subplot(laxs[2, 1])
    vmin = np.percentile(mat, 10)
    vmax = np.percentile(mat, 90)
    sns.heatmap(mat, cmap='viridis', vmin=vmin, vmax=vmax, rasterized=True,
                yticklabels=ggs, xticklabels=False, ax=ax1, cbar_ax=ax2)
    
    for j in range(2):
        ax3 = fig.add_subplot(laxs[j, 0])
        ax3.set_ylim(0, 1)
        ax3.set_xlim(0, adata.n_obs)
        ax3.set_axis_off()
        lns = []
        for k in range(adata.n_obs):
            lns.append([(k+0.5, 0), (k+0.5, 1)])
        if j == 0:
            lns = LineCollection(lns, linewidths=0.1, cmap='viridis', array=adata.obs['ptime'].values)
        else:
            cls = pd.DataFrame(adata.uns['celltype_colors'],
                               index=adata.obs['celltype'].cat.categories).loc[adata.obs['celltype'].values, 0].values
            lns = LineCollection(lns, linewidths=0.1, color=cls)
        ax3.add_collection(lns)
plt.show()